1. перебор сдлеать в виде функции
2. сделать вначале отправку запроса в браузер, а потом сбор json (чтобы навярняка были цены в кеше
3. после получения таблицы сделать алгоритм подбора маршрута с сортировкой от самого дешевого
4. сделать локальный файл-базу в котором будет инфа о том, что я отправлял запрос, и после каждого запуска скрипта, чтобы этот файл чистился (все что старое удалялось)
5. при фильтрации добавить duration (когда сравниваю две даты)
6. реализовать систему весов для сортировки

In [59]:
#!pip install selenium

In [64]:
# https://support.travelpayouts.com/hc/ru/categories/200358578-API-%D0%B8-%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D0%B5
# https://support.travelpayouts.com/hc/ru/articles/203956163-Aviasales-API-%D0%B4%D0%BE%D1%81%D1%82%D1%83%D0%BF%D0%B0-%D0%BA-%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D0%BC-%D0%B4%D0%BB%D1%8F-%D1%83%D1%87%D0%B0%D1%81%D1%82%D0%BD%D0%B8%D0%BA%D0%BE%D0%B2-%D0%BF%D0%B0%D1%80%D1%82%D0%BD%D0%B5%D1%80%D1%81%D0%BA%D0%BE%D0%B9-%D0%BF%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D1%8B#15

In [75]:
import requests
import re
import json

from itertools import product
from tqdm import tqdm

from datetime import datetime
from datetime import timedelta

from time import sleep, strftime
import pandas as pd
from selenium import webdriver

# Используйте тут ваш путь к chromedriver!
chromedriver_path = 'chromedriver.exe'

driver = webdriver.Chrome(executable_path=chromedriver_path) # Этой командой открывается окно Chrome


add_zero = lambda x: '0' + str(x) if len(str(x)) < 2 else str(x)

def range_date(datetime_start, datetime_end):
    delta = timedelta(days = 1)
    range_dates = []
    date = datetime_start
    while date <= datetime_end:
        range_dates.append(date)
        date = date + delta
    
    return range_dates




In [2]:
import json
import codecs
f = codecs.open('airports.json', encoding='utf-8', mode='r')
data = json.load(f)


In [3]:
url = 'http://api.travelpayouts.com/data/routes.json'
response = requests.get(url)
routes = json.loads(response.text)

In [69]:
ACCESS_TOKEN = 'b8848ff67e92768d2d1977862372d796'

origin = 'KWE' #'KWE'
final_destination = 'LED' #'LED' 'KLO'
range_date = ['2020-01-07', '2020-01-15']

time_sleep = 2
date_start, date_end = range_date[0], range_date[1]

In [5]:
destination_list = list(filter(lambda x: x['departure_airport_iata'] == origin, routes))
destination_list = list(map(lambda x: x['arrival_airport_iata'],destination_list))

url = 'http://api.travelpayouts.com/v2/prices/month-matrix'

datetime_start = datetime.strptime(date_start, '%Y-%m-%d')
datetime_end = datetime.strptime(date_end, '%Y-%m-%d')

tickets = []

parameters = list(product(range_date(datetime_start, datetime_end), destination_list))

for datetime_object, destination in tqdm(parameters, total=len(parameters)):
    month, day = add_zero(datetime_object.month), add_zero(datetime_object.day)
    date = datetime_object.strftime('%Y-%m-%d')
    
    url_for_cash = f'https://www.aviasales.ru/search/{origin}{day}{month}{destination}1'
    driver.get(url_for_cash)
    sleep(time_sleep)
    
for datetime_object, destination in tqdm(parameters, total=len(parameters)):
    
    params = {'currency': 'rub', 'show_to_affiliates':'false', 'token' : ACCESS_TOKEN, 'one_way':'true',
            'origin': origin, 'destination': destination, 'month':date}

    response = requests.get(url, params=params)
    
    if 'errors' not in json.loads(response.text)['data']:
        tickets = tickets + json.loads(response.text)['data']
        
        
ponts_list =  list(set(map(lambda x: x['destination'], tickets)))
parameters = list(product(range_date(datetime_start, datetime_end), ponts_list))

tickets2 = []
for datetime_object, point in tqdm(parameters, total=len(parameters)):
    month, day = add_zero(datetime_object.month), add_zero(datetime_object.day)
    date = datetime_object.strftime('%Y-%m-%d')
    
    url_for_cash = f'https://www.aviasales.ru/search/{origin}{day}{month}{destination}1'
    driver.get(url_for_cash)
    sleep(time_sleep)
    
for datetime_object, point in tqdm(parameters, total=len(parameters)):
    
    params = {'currency': 'rub', 'show_to_affiliates':'false', 'token' : ACCESS_TOKEN, 'one_way':'true',
            'origin': point, 'destination': final_destination, 'month':date}

    response = requests.get(url, params=params)
    
    if 'errors' not in json.loads(response.text)['data']:
        tickets2 = tickets2 + json.loads(response.text)['data']
        
df = pd.DataFrame(tickets + tickets2)

feature = list(df.columns)
feature.remove('found_at')

df.sort_values(by = 'found_at', ascending=False, inplace=True)
df.drop_duplicates(inplace = True, keep = 'first', subset = feature)

df.to_csv('routes_price.txt', index = False)


100%|████████████████████████████████████████████████████████████████████████████████| 324/324 [01:16<00:00,  3.08it/s]


In [73]:
route_feature = ['depart_date', 'origin', 'destination', 'distance', 
                 'duration', 'gate', 'number_of_changes', 'value']
df_route = df[route_feature].merge(df[route_feature], 
                                   how = 'left', left_on = 'destination', 
                                   right_on = 'origin', suffixes = ['_from', '_to'])

df_route = df_route[df_route['depart_date_from'] <= df_route['depart_date_to']]

value_list = list(filter(lambda x: x[:5] == 'value', list(df_route.columns)))
df_route['total_value'] =  df_route[value_list].sum(axis = 1)

df_route = df_route[(df_route['depart_date_from'] >= date_start) & (df_route['depart_date_to'] <= date_end)]

df_route.sort_values(by = ['total_value', 'depart_date_from'] , inplace = True)

In [74]:
df_route.head(20)

,depart_date_from,origin_from,destination_from,distance_from,duration_from,gate_from,number_of_changes_from,value_from,depart_date_to,origin_to,destination_to,distance_to,duration_to,gate_to,number_of_changes_to,value_to,total_value
1742,2020-01-07,KWE,SHA,1521,130.0,OneTwoTrip,0,2548.0,2020-01-09,SHA,LED,7112.0,1630.0,SuperSaver.ru,1.0,21883.0,24431.0
1057,2020-01-08,KWE,SHA,1521,130.0,OneTwoTrip,0,2548.0,2020-01-09,SHA,LED,7112.0,1630.0,SuperSaver.ru,1.0,21883.0,24431.0
529,2020-01-09,KWE,SHA,1521,130.0,OneTwoTrip,0,2548.0,2020-01-09,SHA,LED,7112.0,1630.0,SuperSaver.ru,1.0,21883.0,24431.0
1749,2020-01-07,KWE,SHA,1521,130.0,OneTwoTrip,0,2548.0,2020-01-13,SHA,LED,7112.0,1560.0,SuperSaver.ru,1.0,21949.0,24497.0
1064,2020-01-08,KWE,SHA,1521,130.0,OneTwoTrip,0,2548.0,2020-01-13,SHA,LED,7112.0,1560.0,SuperSaver.ru,1.0,21949.0,24497.0
536,2020-01-09,KWE,SHA,1521,130.0,OneTwoTrip,0,2548.0,2020-01-13,SHA,LED,7112.0,1560.0,SuperSaver.ru,1.0,21949.0,24497.0
1750,2020-01-07,KWE,SHA,1521,130.0,OneTwoTrip,0,2548.0,2020-01-13,SHA,LED,7112.0,2645.0,KupiBilet.ru,2.0,23581.0,26129.0
1065,2020-01-08,KWE,SHA,1521,130.0,OneTwoTrip,0,2548.0,2020-01-13,SHA,LED,7112.0,2645.0,KupiBilet.ru,2.0,23581.0,26129.0
537,2020-01-09,KWE,SHA,1521,130.0,OneTwoTrip,0,2548.0,2020-01-13,SHA,LED,7112.0,2645.0,KupiBilet.ru,2.0,23581.0,26129.0
1457,2020-01-07,KWE,HGH,1396,125.0,OneTwoTrip,0,4644.0,2020-01-15,HGH,LED,7149.0,760.0,Trip.com,1.0,21599.0,26243.0
